# Working with data

Today our focus will be on data preparation and exploration using flights14 data, mostly using data.table and tidyverse suite

In [ ]:
library(data.table)
library(tidyverse)
library(lubridate)
library(plotly)

Let's load our data:

In [ ]:
flights <- fread("../data/csv/flights14.csv")

And view our data:

In [ ]:
flights

**EXERCISE 1:**

Now just looking at the data, what can you say?

What will be the goal of our final model?

What transformations can we make in existing fields in order to get more meaningful insight?

What new fields can we calculate from existing fields?

Are there any redundant fields?

What interactions can we test for? Can we make some quick and dirty summarizations and visualizations for that

Think about it for several minutes and list your findings

**MY OWN FINDINGS:**

In [ ]:
pass <- readline(prompt = "Please enter the password for the solution: ")
solution <- system(sprintf("cat findings | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:%s 2> /dev/null", pass), intern = T, ignore.stderr = T)
cat(solution, sep = "\n")
#eval(parse(text = solution))

# Handling data and time

## Creating a new date variable

For handling time and date data, we will be using the lubridate package from tidyverse.

We have year, month and day. We can paste them into a date string and parse as a date object:

In [ ]:
?lubridate::dmy

In [ ]:
paste(1, 2, 2014, sep = "-") %>% lubridate::dmy()

Or

In [ ]:
paste(2014, 2, 1, sep = "-") %>% lubridate::ymd()

In [ ]:
head(flights)

The workflow be as follows:

- Using "unite" function from tidyr, we will paste three columns (year, month, day) with "-" as separators. So 2014, 1 and 1 will be 2014-1-1
- Using "ymd" function from lubridate we will parse the date strings as date objects.
- The resulting date value is stored as a numeric value (the number of days since 1970-01-01) in data table. We will convert it to date class
- The resulting date object is saved as a new column called "date" inside the data.table

In [ ]:
flights[,date := tidyr::unite(.SD, sep = "-") %>%
                    sapply(lubridate::ymd) %>%
                    as.Date(origin = "1970-01-01"),
        .SDcols = year:day]

In [ ]:
flights

## creating a new "dep_minutes" variable

We will be using "hm" function from the lubridate package to create a "period" object:

In [ ]:
hm("9:14") %>% class()

By taking modulo and floor divisio we can convert the numeric representation to minutes:

In [ ]:
flights[, dep_minutes := .(dep_time %/% 100 * 60 + dep_time %% 100)]

In [ ]:
flights

## create datetime field

In fact UNIX systems does not have separate data types for date or time. When we first create a date object, it is in fact a datetime object at midnight:

In [ ]:
date1 <- as.Date("2014-01-01") %>% as.character() %>% as.POSIXct()
date1

"1" means a second in POSIXct format and a day in Date format. Let's add 6 hours and 35 minutes to the POSIXct formatted datatime object

In [ ]:
date1 + (6 * 3600 + 35 * 60)

It would be useful to create a separate field for the departure date time

In [ ]:
flights[, dep_datetime := date %>% as.character() %>% as.POSIXct() + dep_minutes * 60]

In [ ]:
flights

## Create datahour field

And a separate field that combines the date and hour (w/o minutes) would also be be useful for grouping purposes:

In [ ]:
flights[, dep_datehour := (as.numeric(dep_datetime) %/% 3600 * 3600) %>% as.POSIXct(origin = "1970-01-01")]

In [ ]:
flights

## Create weekday field

We can code dates as the weekdays using lubridate package:

In [ ]:
flights[, weekday := lubridate::wday(date, label = T, abbr = T, locale = "en_US.UTF-8")]

In [ ]:
flights

## Other variables to be created

Now, we may need the air_speed in miles per hour: 

In [ ]:
flights[, air_speed := distance / air_time * 60]

In [ ]:
flights

And we can create a variable for "gain" as the difference between departure delay and arrival delay:

In [ ]:
flights[, gain := dep_delay - arr_delay]

In [ ]:
flights

## Variables to be deleted

We may not talk about delays in cancelled flights. Let's see the cancelled flights:

In [ ]:
flights[cancelled != 0]

In [ ]:
flights[,unique(cancelled)]

No information inside cancelled variable, as also the case with year variable (all in 2014). Let's delete them:

In [ ]:
flights[,c("year", "cancelled") := NULL]

In [ ]:
flights

We can keep the hour variable and see the hour of the day effect but minute is too granular so we can also delete it:

In [ ]:
flights[, minute := NULL]

## Factors

Now let's view the data again and the structure:

In [ ]:
flights

In [ ]:
str(flights)

The weekday is created as an ordinal factor (with Sunday as the starting point).

However we may convert character variables to factors:

In [ ]:
factor_cols <- flights[,lapply(.SD, is.character) %>% unlist()]
factor_cols

for (col in names(flights)[factor_cols])
set(flights, j = col, value = as.factor(flights[[col]]))

In [ ]:
flights

In [ ]:
str(flights)

# Summarize and explore data

## Delays by origin, day and weekday

Let's summarize the dep_delay by origin and date:

In [ ]:
flights_delay_date_origin <- flights[,.(dep_delay_mean = mean(dep_delay), weekday = weekday), by = c("origin", "date")]

In [ ]:
flights_delay_date_origin

And let's create a line plot of mean dep_delay for each origin across dates

Note that we first create the object with ggplot and then convert into plotly to get the benefits of plotly's interactivity (such as tooltip over hover):

In [ ]:
g <- flights_delay_date_origin %>%
    ggplot(aes(x = date,
          y = dep_delay_mean,
          color = origin)) +
geom_line()

plotly::ggplotly(g)

There is an outlier period in the first week of January. We will try to understand the reason

Now let's summarize the data in another way: Insted of average dep_delays, let's get daily flight counts for each origin and date and visualize the same way:

In [ ]:
flights_count_date_origin <- flights[,.(flight_count = .N), by = c("origin", "date", "weekday")]

In [ ]:
flights_count_date_origin

Let's plot the values on lines:

In [ ]:
g <- flights_count_date_origin %>%
    ggplot(aes(x = date,
          y = flight_count,
          color = origin)) +
geom_line()

plotly::ggplotly(g)

The peak in delays is not related to flight counts. But we see a short term periodicity with the flight counts.

Does it have anything to do with the weekday effect?

In [ ]:
g <- flights_count_date_origin %>%
    ggplot(aes(x = date,
          y = flight_count,
          color = weekday)) +
geom_line() +
facet_wrap(~ origin, ncol = 3)

plotly::ggplotly(g)

Saturdays have lower flight counts 

## Hourly counts vs. delays

However, daily basis is to broad to depict the relationship between counts and delays: delays may not be persistent daylong

Let's plot average delays and counts on a hourly basis:

In [ ]:
flights

In [ ]:
flights_count_delay_hourly <- flights[,.(flight_count = .N, mean_dep_delay = mean(dep_delay)), by = c("origin", "dep_datehour")]

In [ ]:
flights_count_delay_hourly

In [ ]:
g <- flights_count_delay_hourly %>%
    ggplot(aes(x = flight_count,
              y = mean_dep_delay,
              color = origin)) +
geom_point()

plotly::ggplotly(g)

There is more variation in mean_dep_delay and the overall level of mean_dep_delay is higher with lower flight counts. A surprising finding!

Maybe we should look for the answer in the destination not origin?

**IN-CLASS CHALLANGE**

In [ ]:
pass <- readline(prompt = "Please enter the password for the solution: ")
solution <- system(sprintf("cat encrypt | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:%s 2> /dev/null", pass), intern = T, ignore.stderr = T)
cat(solution, sep = "\n")
#eval(parse(text = solution))